## Training SVM classifier on DD_ACC dataset

The first step is to read dataset from CSV file

In [26]:
import pandas as pd
from sklearn import preprocessing


#dd_dataset = pd.read_csv('./dataset/dd_pssm_dataset_improved.csv')
# Add separated dimers feature extraction

dd_dataset = pd.read_csv('../dataset/EDD/EDD_weighted_IG_feature_900.csv')
#dd_dataset_test = pd.read_csv('../dataset/F86/dd_fusion_acc_dimer_200.csv')
#dd_dataset=dd_dataset.groupby(["Fold"]).head(16)


In [27]:
dd_dataset.head()

,Fold,Protein name,F1740,F1980,F1631,F1962,F1862,F1993,F1722,F1634,...,F693,F514,F1244,F458,F228,F242,F229,F1167,F1306,F757
0,b.40\t-\tOB-fold,d1b8aa1b,2707.084539,2728.557287,2907.499367,2771.288129,2675.009857,2478.355373,2749.815382,2683.832082,...,-0.242851,-0.521487,3.025706e-16,-0.204792,0.193311,-0.067750,0.978465,0.304763,-0.022662,0.050752
1,b.40\t-\tOB-fold,d2je6i1b,1163.503546,1219.989459,1326.333499,1346.218466,1388.843348,924.222638,1361.912122,1388.843348,...,-0.171151,0.520564,4.562682e-17,-0.653828,-0.836504,-0.185059,-0.170849,-0.707552,-0.083094,0.641716
2,b.40\t-\tOB-fold,d3d3ra1b,1546.011015,1766.844258,1521.198894,1866.072526,1546.953598,1654.144423,1645.239284,1534.156156,...,-0.079669,-0.537916,-1.162916e-01,0.353436,-0.617565,0.398900,1.157175,0.348609,-0.152135,-0.661761
3,b.40\t-\tOB-fold,d1jmca1b,3216.722008,3103.645324,3217.954949,3216.722008,3103.645324,2990.568640,3329.798692,3202.990918,...,-0.269738,-0.082844,-7.371818e-02,-0.328803,-0.593586,-0.135128,-0.634494,-0.267282,-0.110450,-0.625863
4,b.40\t-\tOB-fold,d1jjga_b,565.235082,604.082491,694.839846,714.263550,694.839846,548.585398,675.416141,682.509050,...,-0.093865,-0.136241,-3.384136e-02,-0.126344,-0.142135,-0.038632,-0.114255,-0.197061,-0.001878,-0.096625


Convert class lables to unique integers

In [28]:
# Encode class labels
#from sklearn.preprocessing import LabelEncoder
dd_dataset=dd_dataset.loc[dd_dataset['Fold'] != 'P-loopcontainingnucleosidetriphosphatehydrolases(c.37)']
y_true, labels = pd.factorize(dd_dataset.Fold)
dd_dataset.insert(1, 'class labels', y_true)

#dd_dataset_test=dd_dataset_test.loc[dd_dataset_test['Fold'] != 'P-loopcontainingnucleosidetriphosphatehydrolases(c.37)']
#y_true_test, labels_test = pd.factorize(dd_dataset_test.Fold)
#dd_dataset_test.insert(1, 'class labels', y_true_test)



In [29]:
dd_dataset.head()


,Fold,class labels,Protein name,F1740,F1980,F1631,F1962,F1862,F1993,F1722,...,F693,F514,F1244,F458,F228,F242,F229,F1167,F1306,F757
0,b.40\t-\tOB-fold,0,d1b8aa1b,2707.084539,2728.557287,2907.499367,2771.288129,2675.009857,2478.355373,2749.815382,...,-0.242851,-0.521487,3.025706e-16,-0.204792,0.193311,-0.067750,0.978465,0.304763,-0.022662,0.050752
1,b.40\t-\tOB-fold,0,d2je6i1b,1163.503546,1219.989459,1326.333499,1346.218466,1388.843348,924.222638,1361.912122,...,-0.171151,0.520564,4.562682e-17,-0.653828,-0.836504,-0.185059,-0.170849,-0.707552,-0.083094,0.641716
2,b.40\t-\tOB-fold,0,d3d3ra1b,1546.011015,1766.844258,1521.198894,1866.072526,1546.953598,1654.144423,1645.239284,...,-0.079669,-0.537916,-1.162916e-01,0.353436,-0.617565,0.398900,1.157175,0.348609,-0.152135,-0.661761
3,b.40\t-\tOB-fold,0,d1jmca1b,3216.722008,3103.645324,3217.954949,3216.722008,3103.645324,2990.568640,3329.798692,...,-0.269738,-0.082844,-7.371818e-02,-0.328803,-0.593586,-0.135128,-0.634494,-0.267282,-0.110450,-0.625863
4,b.40\t-\tOB-fold,0,d1jjga_b,565.235082,604.082491,694.839846,714.263550,694.839846,548.585398,675.416141,...,-0.093865,-0.136241,-3.384136e-02,-0.126344,-0.142135,-0.038632,-0.114255,-0.197061,-0.001878,-0.096625


Create a training set from pandas data frame

In [30]:
train_data = dd_dataset.iloc[:,3:].values

# Normalize dataset
min_max_scaler = preprocessing.MinMaxScaler()
train_scaled = min_max_scaler.fit_transform(train_data)
train_data = train_scaled
#test_data = dd_dataset_test.iloc[:,3:].values
#test_scaled_test = min_max_scaler.fit_transform(test_data)
#test_data = test_scaled_test

print("Number of samples: %d, Number of features: %d" % (train_data.shape[0], train_data.shape[1]))

Number of samples: 432, Number of features: 900


In [31]:
# Show normlaized data

train = pd.DataFrame(train_data)
#test = pd.DataFrame(test_data)

train.head()

,0,1,2,3,4,5,6,7,8,9,...,890,891,892,893,894,895,896,897,898,899
0,0.200876,0.199713,0.208755,0.196581,0.197680,0.190778,0.196623,0.198332,0.198217,0.208864,...,0.203246,0.325558,0.320329,0.255444,0.431877,0.289140,0.719140,0.397383,0.320871,0.451874
1,0.086336,0.089295,0.095229,0.095494,0.102634,0.071145,0.097382,0.102634,0.088885,0.095279,...,0.234679,0.612462,0.320329,0.086731,0.133076,0.241583,0.386960,0.182719,0.288694,0.596638
2,0.114720,0.129322,0.109220,0.132370,0.114318,0.127332,0.117641,0.113372,0.128278,0.120547,...,0.274783,0.321034,0.288410,0.465184,0.196601,0.478320,0.770791,0.406680,0.251933,0.277334
3,0.238693,0.227167,0.231046,0.228178,0.229356,0.230207,0.238094,0.236697,0.241373,0.231167,...,0.191459,0.446328,0.300095,0.208851,0.203559,0.261825,0.252955,0.276079,0.274128,0.286128
4,0.041943,0.044215,0.049889,0.050666,0.051348,0.042229,0.048295,0.050437,0.047971,0.049915,...,0.268559,0.431626,0.311040,0.284919,0.334547,0.300945,0.403317,0.290970,0.331937,0.415772


## Data analysis

In [32]:
# Frequency of folds
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
freq_plot = dd_dataset['Fold'].value_counts().plot(ax=ax, kind='bar')
freq_plot.set_ylabel('Number of proteins')
#plt.show()

Text(0,0.5,'Number of proteins')

## SVM classifier

In [33]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Hyperparameters
kernel = 'rbf'

# An instance of SVM classifier
svm_cl = SVC(kernel=kernel)

Define range of parameters for Grid Search

In [34]:
# Penalty parameter
c_range = {'C': [float(2**i) for i in range(-14, 14)]}
# Gamma parameter for RBF kernel
gamma_range = {'gamma': [float(2**i) for i in range(-14, 14)]} if kernel == 'rbf' else {}

param_range = {**c_range, **gamma_range}

# Arguments for grid search
cv_fold = 10
n_workers = 5 # Number of CPU threads

result = GridSearchCV(svm_cl, param_range, cv=cv_fold, n_jobs=n_workers, refit=True,
                      verbose=1)

Start grid search!

In [35]:
result.fit(train_data, y_true)

Fitting 10 folds for each of 784 candidates, totalling 7840 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    7.6s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:   22.8s
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:   48.4s
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed:  1.4min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed:  2.2min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed:  3.4min
[Parallel(n_jobs=5)]: Done 2440 tasks      | elapsed:  4.9min
[Parallel(n_jobs=5)]: Done 3190 tasks      | elapsed:  6.6min
[Parallel(n_jobs=5)]: Done 4040 tasks      | elapsed:  8.4min
[Parallel(n_jobs=5)]: Done 4990 tasks      | elapsed: 10.4min
[Parallel(n_jobs=5)]: Done 6040 tasks      | elapsed: 12.7min
[Parallel(n_jobs=5)]: Done 7190 tasks      | elapsed: 15.3min
[Parallel(n_jobs=5)]: Done 7840 out of 7840 | elapsed: 16.8min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=5,
       param_grid={'gamma': [6.103515625e-05, 0.0001220703125, 0.000244140625, 0.00048828125, 0.0009765625, 0.001953125, 0.00390625, 0.0078125, 0.015625, 0.03125, 0.0625, 0.125, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0, 1024.0, 2048.0, 4096.0, 8192.0], 'C': [6.103515625e-05, 0.0...25, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0, 1024.0, 2048.0, 4096.0, 8192.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

Confusion matrix

In [36]:
    print("Best accuracy: %.2f" % (result.best_score_ * 100))

Best accuracy: 23.84


Classification report

In [37]:
import sys
sys.path.insert(0, '../')


from misc import plt_confusion_matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np

y_pred = result.best_estimator_.predict(train_data)

cm = confusion_matrix(labels[y_true], labels[y_pred])

print("Accuracy: ", (accuracy_score(labels[y_true], labels[y_pred]) * 100))

plt.figure(figsize=(20, 15))
plt_confusion_matrix(cm, np.unique(labels[y_true]))
plt.savefig("../report_final/EDD_weighted_IG_feature_900.png", dpi=256)

Accuracy:  91.20370370370371


In [38]:
print(classification_report(labels[y_true], labels[y_pred]))
def classification_report_csv(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        row['class'] = row_data[len(row_data)-5]
        row['precision'] = float(row_data[len(row_data)-4])
        row['recall'] = float(row_data[len(row_data)-3])
        row['f1_score'] = float(row_data[len(row_data)-2])
        row['support'] = float(row_data[len(row_data)-1])
        report_data.append(row)
    dataframe = pd.DataFrame.from_dict(report_data)
    dataframe.to_csv("../report_final/EDD_weighted_IG_feature_900.csv", index = False)
    
classification_report_csv(classification_report(labels[y_true], labels[y_pred]))

                                                           precision    recall  f1-score   support

                                        a.1	-	Globin-like       1.00      1.00      1.00        16
                     a.24	-	Four-helicalup-and-downbundle       1.00      0.81      0.90        16
                                a.26	-	4-helicalcytokines       1.00      1.00      1.00        16
                                        a.3	-	Cytochromec       1.00      1.00      1.00        16
                                       a.39	-	EFHand-like       1.00      0.88      0.93        16
                     a.4	-	DNA/RNA-binding3-helicalbundle       1.00      0.88      0.93        16
                   b.1	-	Immunoglobulin-likebeta-sandwich       1.00      0.75      0.86        16
b.121	-	Nucleoplasmin-like/VP(viralcoatandcapsidproteins)       1.00      0.88      0.93        16
              b.29	-	ConcanavalinA-likelectins/glucanases       1.00      0.81      0.90        16
         